_La base des TDs pour le cours "Classification des données" a été prise du cours en ligne "Open Machine Learning Course" (https://mlcourse.ai/, __auteur Yury Kashnitsky__)_ 

# <center> TD 1. Analyse exploratoire et analyse visuelle de données avec la librairie Pandas et Seaborn

## 1. Mise en pratique des principales méthodes de Pandas

**[Pandas](http://pandas.pydata.org)** est une bibliothèque Python qui fournit des moyens étendus pour l’analyse de données. Les Data scientistes travaillent souvent avec des données stockées dans des formats sous forme de table de données telles que `.csv`,` .tsv` ou `.xlsx`. Pandas est très pratique pour charger, traiter et analyser ces données tabulaires à l’aide de requêtes quasi-similaires aux requêtes de type SQL. En complément de `Matplotlib` et` Seaborn`, `Pandas` offre un large éventail d'opportunités d'analyse visuelle des données tabulaires.

Les pricipales structures de données dans `Pandas` sont implémentées avec les classes **Series** et **DataFrame**. Le premier est un tableau unidimensionnel indexé d'un type de données fixe. Le second est une structure de données bi-dimensionnelle - une table - dans laquelle chaque colonne contient des données du même type. Vous pouvez la voir comme un dictionnaire de plusieurs `Series`. Les `DataFrames` sont parfaits pour représenter des données réelles : les lignes correspondent aux instances (exemples, observations, individus etc.) et les colonnes correspondent aux caractéristiques de ces instances (variables).

Vous pouvez trouver tous les commandes utiles de Pandas dans le fichier "Pandas Cheat Sheet" (https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)

# quelques imports pour mettre en place le cadre du graphique 
import matplotlib.pyplot as plt

# !pip install seaborn  #(pour installer la librairie seaborn via le notebook)
import seaborn as sns

# import de paramètres pour améliorer le rendu visuel
sns.set()
# Les graphiques au format Retina sont plus nets et plus lisibles
%config InlineBackend.figure_format = 'retina'

Nous allons tester les principales méthodes en analysant un jeu de données ou [dataset](https://bigml.com/user/francisco/gallery/dataset/5163ad540c0b5e5b22000383) sur le taux de désabonnement des clients d'opérateurs téléphoniques. Chargons les données (en utilisant la méthode `read_csv`), et jetons un coup d’œil aux 5 premières lignes en utilisant la méthode` head`:

In [ ]:
url = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/telecom_churn.csv"
df = pd.read_csv(url)

df.head()

Rappelez-vous que chaque ligne correspond à un client, à une **instance** et les colonnes sont les **caractéristiques** de cette instance.

Examinons la dimensionnalité des données, les noms des caractéristiques et les types de caractéristiques.

In [ ]:
print(df.info())

`bool`,` int64`, `float64` et` object` sont les types de données de nos caractéristiques. Nous voyons qu'une caractéristique est logique (`bool`), 3 caractéristiques sont de type ` objet`, et 16 caractéristiques sont numériques. Avec cette même méthode, nous pouvons facilement voir s’il manque des valeurs. Ici, il n'y en a pas car chaque colonne contient 3333 observations.

Nous pouvons **changer le type de colonne** avec la méthode `astype`. Appliquons cette méthode à la caractéristique `Churn` pour la convertir en` int64`:

In [ ]:
df['Churn'] = df['Churn'].astype('int64')

La méthode `describe` affiche les caractéristiques statistiques de base de chaque caractéristique numérique (type ` int64` et `float64`): nombre de valeurs non manquantes, moyenne, écart-type, amplitude, médiane, (1er : 0,25) et (3ème : 0,75) quartiles.

In [ ]:
df.describe()

Afin de voir les statistiques des caractéristiques non numériques, il faut indiquer explicitement ces types de données dans le paramètre `include`.

In [ ]:
df.describe(include=['object', 'bool'])

Pour les caractéristiques catégorielles (type `objet`) et booléennes (type` bool`), nous pouvons utiliser la méthode `value_counts`. Jetons un coup d'oeil à la distribution de `Churn` (la caractéristique cible pour cette base des données) :

In [ ]:
df['Churn'].value_counts()

2850 utilisateurs sur 3333 sont clients *fidèles*; leur valeur `Churn` est 0. Pour calculer les pourcentages, passez ` normalize = True` à la fonction `value_counts`.

In [ ]:
df['Churn'].value_counts(normalize=True)

Ou avec la visualisation :

In [ ]:
df['Churn'].value_counts().plot(kind='bar', label='Churn')
plt.legend()
plt.title('Répartition de désabonnement de clients');

### Tri

Un `DataFrame` peut être trié selon la valeur de l’une des variables (colonnes). Par exemple, nous pouvons trier par *Total day charge*  
(utilisez `ascending = False` pour trier par ordre décroissant):

In [ ]:
df.sort_values(by='Total day charge', ascending=False).head()

Nous pouvons également trier sur plusieurs colonnes:

In [ ]:
df.sort_values(by=['Churn', 'Total day charge'], ascending=[True, False]).head()

### Indexation et récupération de données

Un `DataFrame` peut être indexé de différentes manières.

Pour obtenir une seule colonne, vous pouvez saisir : `DataFrame['NomDeColonne'] `. Que nous utilisons pour répondre à une question à propos de cette colonne uniquement: **quelle est la proportion d'utilisateurs qui se sont désabonnés dans notre base de données?**

In [ ]:
df['Churn'].mean()

14,5% est en fait assez mauvais pour une entreprise; un tel taux de désabonnement peut entraîner la faillite de l'entreprise.

**L'indexation booléenne** avec une colonne est également très pratique. La syntaxe est `df[P(df['Nom']]]`, où `P` est une condition logique vérifiée pour chaque élément de la colonne ` NomDeColonne`. Le résultat de cette indexation est le `DataFrame` composé uniquement de lignes satisfaisant la condition ` P` de la colonne `NomDeColonne`.

Exemple d'utlisation pour répondre à la question:

**Quelles sont les valeurs moyennes des caractéristiques numériques pour les utilisateurs désabonnés, c'est-à-dire qui ont un Churn égal à 1 ?**

In [ ]:
df[df['Churn'] == 1].mean()

**Combien de temps (en moyenne) les utilisateurs désabonnés passent-ils au téléphone pendant la journée?**

In [ ]:
df[df['Churn'] == 1]['Total day minutes'].mean()

**Quelle est la durée maximale des appels internationaux parmi les clients fidèles (`Churn == 0`) n'ayant pas de forfait international?**

In [ ]:
df[(df['Churn'] == 0) & (df['International plan'] == 'No')]['Total intl minutes'].max()

Les DataFrames peuvent être indexés par nom de colonne (étiquette) ou nom de ligne (index) ou par le numéro de série (indice) d'une ligne. La méthode `loc` est utilisée pour **l'indexation par nom**, tandis que` iloc() `est utilisée pour **l'indexation par numéro**.

Dans le premier cas ci-dessous, nous *"récupérons les valeurs des lignes d'index de 0 à 5 (inclus) et des colonnes étiquetées de  State à Area code (inclus)"*.  
Dans le second cas, nous *"récupérons les valeurs des cinq premières lignes des trois premières colonnes"* (comme dans un slicing avec Python : la valeur maximale n'est pas incluse).

In [ ]:
df.loc[0:5, 'State':'Area code']

In [ ]:
df.iloc[0:5, 0:3]

Si nous avons besoin de la première ou de la dernière ligne du dataframe, nous pouvons utiliser la syntaxe : `df[:1]` ou `df[-1:]`:

In [ ]:
df[-1:]

### Application de fonctions à des cellules, des colonnes et des lignes

**Pour appliquer des fonctions à chaque colonne, utilisez `apply ()`:**

In [ ]:
df.apply(np.max) 

La méthode `apply` peut également être utilisée pour appliquer une fonction à chaque ligne. Pour ce faire, spécifiez `axis = 1`. Les fonctions Lambda sont très pratiques dans de tels scénarios. Par exemple, si nous devons sélectionner tous les _state_ commençant par 'W', nous pouvons le faire comme suit:

In [ ]:
df[df['State'].apply(lambda state: state[0] == 'W')].head()

La méthode `map` peut être utilisée pour **remplacer des valeurs dans une colonne** en transmettant un dictionnaire de la forme` {ancienne_valeur: nouvelle_valeur} `comme argument:

In [ ]:
d = {'No' : False, 'Yes' : True}
df['International plan'] = df['International plan'].map(d)
df.head()

Presque la même chose peut être faite avec la méthode `replace`.

<details>

<p>
Il y a une petite différence.  
La méthode `replace` ne fera rien avec des valeurs qui ne se trouvent pas dans le dictionnaire de mappage,  
alors que `map` les changera en `NaN`).  
<br>    
    
```python
a_series = pd.Series(['a', 'b', 'c'])
a_series.replace({'a': 1, 'b': 1}) # 1, 2, c
a_series.map({'a': 1, 'b': 2}) # 1, 2, NaN
```
</p>
</details>

In [ ]:
df = df.replace({'Voice mail plan': d})
df.head()

### Agrégation de données

En général, le regroupement des données dans Pandas fonctionne comme suit :


```python
df.groupby(by=grouping_columns)[columns_to_show].function()
```

1. Premièrement, la méthode `groupby` divise les colonnes`grouping_columns` par leurs valeurs. Ils deviennent un nouvel index dans le dataframe qui en  résulte.
2. Ensuite, les colonnes choisies sont sélectionnées (`columns_to_show`). Si `columns_to_show` n'est pas inclus, toutes les clauses non groupby seront incluses.
3. Enfin, une ou plusieurs fonctions sont appliquées aux groupes obtenus par colonnes sélectionnées.

Voici un exemple où nous regroupons les données en fonction des valeurs de la variable `Churn` et affichons les statistiques de trois colonnes dans chaque groupe :

In [ ]:
columns_to_show = ['Total day minutes', 
                   'Total eve minutes', 
                   'Total night minutes']

df.groupby(['Churn'])[columns_to_show].describe(percentiles=[])

Faisons la même chose, mais légèrement différemment, en passant une liste de fonctions à `agg ()` (agrégateur) :

In [ ]:
columns_to_show = ['Total day minutes', 
                   'Total eve minutes', 
                   'Total night minutes']

df.groupby(['Churn'])[columns_to_show].agg([np.mean, np.std, np.min, np.max])

### Tableaux récapitulatifs

Supposons que nous voulions voir comment les observations de notre ensemble de données sont réparties dans le contexte de deux variables - `Churn` et`International plan`. Pour ce faire, nous pouvons construire un **tableau de contingence** en utilisant la méthode `crosstab`:

In [ ]:
pd.crosstab(df['Churn'], df['International plan'])

In [ ]:
pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)

Nous pouvons constater que la plupart des utilisateurs sont fidèles et n'utilisent pas de services supplémentaires (International Plan/Voice mail).

Cela ressemblera aux **tableaux croisés dynamiques** pour ceux qui connaissent Excel. Et, bien sûr, les tableaux croisés dynamiques sont implémentés dans Pandas: la méthode `pivot_table` prend les paramètres suivants:

* `values` - une liste de variables pour calculer des statistiques,  
* `index` - une liste de variables pour regrouper les données,
* `aggfunc` - quelles statistiques nous devons calculer pour les groupes, ex. somme, moyenne, maximum, minimum ou autre chose.

Examinons le nombre moyen d'appels de jour, de soir et de nuit par code régional (area code):

In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'],
               ['Area code'], aggfunc='mean')

### Opérations de transformations d'un DataFrame

Comme beaucoup d'autres choses avec Pandas, l'ajout de colonnes à un DataFrame est réalisable de plusieurs manières.

Par exemple, si nous voulons calculer le nombre total d'appels pour tous les utilisateurs, créons la série `total_calls` et collons-la dans le DataFrame:

In [ ]:
total_calls = df['Total day calls'] + df['Total eve calls'] + \
              df['Total night calls'] + df['Total intl calls']
df.insert(loc=len(df.columns), column='Total calls', value=total_calls) 
# le paramètre loc est le nombre de colonnes après lequel l'objet Série doit être inséré
# nous l'initialisons à len(df.columns) pour le coller à la toute fin du dataframe
df.head()

Il est possible d’ajouter une colonne plus facilement sans créer d’instance Series intermédiaire:

In [ ]:
df['Total charge'] = df['Total day charge'] + df['Total eve charge'] + \
                     df['Total night charge'] + df['Total intl charge']
df.head()

Pour supprimer des colonnes ou des lignes, utilisez la méthode `drop`, en passant les index requis et le paramètre` axis` (`1` si vous supprimez des colonnes et rien ou` 0` si vous supprimez des lignes). L'argument `inplace` indique s'il faut modifier le DataFrame d'origine. Avec `inplace = False`, la méthode` drop` ne modifie pas le DataFrame existant et en renvoie un nouveau avec des lignes ou des colonnes supprimées. Avec `inplace = True`, il modifie le DataFrame.

In [ ]:
# supprimer les colonnes qui viennent d'être créées
df.drop(['Total charge', 'Total calls'], axis=1, inplace=True) 
# et voici comment supprimer des lignes
df.drop([1, 2]).head() 

## 2. Prévision du churn (taux d'attrition)

Voyons comment le taux de désabonnement est lié à la caractéristique ou variable *International plan*. Pour ce faire, nous utiliserons un tableau de contingence `` crosstab`` et également une analyse visuelle avec `Seaborn` (`sns`).

In [ ]:
pd.crosstab(df['Churn'], df['International plan'], margins=True)

In [ ]:
sns.countplot(x='International plan', hue='Churn', data=df);

Nous voyons qu'avec *International Plan*, le taux de désabonnement est beaucoup plus élevé, ce qui est une observation intéressante! Peut-être des dépenses importantes et mal contrôlées avec des appels internationaux sont-elles très sujettes aux conflits et suscitent l’insatisfaction des clients de l’opérateur de télécommunications.

Voyons ensuite une autre fonctionnalité importante - *Customer service calls*. Faisons également un tableau de synthèse et une image.

In [ ]:
pd.crosstab(df['Churn'], df['Customer service calls'], margins=True)

In [ ]:
sns.countplot(x='Customer service calls', hue='Churn', data=df);

Bien que ce ne soit pas évident dans le tableau récapitulatif, il ressort clairement du graphique ci-dessus que le taux de résiliation augmente fortement à partir de 4 appels de service après-vente.

Ajoutons maintenant une variable binaire à notre DataFrame - `Customer service calls > 3` (Appels du service client> 3). Et encore une fois, voyons comment cela se rapporte au désabonnement.

In [ ]:
df['Many_service_calls'] = (df['Customer service calls'] > 3).astype('int')

pd.crosstab(df['Many_service_calls'], df['Churn'], margins=True)

In [ ]:
sns.countplot(x='Many_service_calls', hue='Churn', data=df);

Construisons une autre table de contingence qui relie *Churn* à la fois à *International plan* et à la variable nouvellement créée *Many_service_calls*.

In [ ]:
pd.crosstab(df['Many_service_calls'] & df['International plan'] , df['Churn'])

Par conséquent, si un nombre d'appels vers le centre de services est supérieur à 3 et que le *International Plan* est ajouté (et en prédisant Churn=0 sinon), on peut s’attendre à une précision de 85,8%. Ce nombre, 85,8%, que nous avons obtenu grâce à ce raisonnement très simple constitue un bon point de départ (*référence*) pour les autres modèles d’apprentissage automatique que nous allons construire.

Au cours de ce cours, rappelez-vous qu'avant l'avènement de l'apprentissage automatique, le processus d'analyse des données ressemblait à ce que nous venons de réaliser. Récapitulatif :
    
- La part des clients fidèles dans l'ensemble de données est de 85,5%. Le modèle le plus "simple" qui prédit toujours un "client fidèle" sur de telles données devinera juste dans environ 85,5% des cas. C'est-à-dire que la proportion de réponses correctes (*précision*) des modèles suivants ne devrait pas être inférieure à ce nombre et qu'elle devrait être nettement supérieure;
- A l’aide d’une simple prévision pouvant être exprimée par la formule suivante: `International plan = True & Customer Service calls > 3 => Churn = 1, else Churn = 0`, on peut s’attendre à un taux de prédiction de 85,8%, qui est juste au-dessus de 85,5%. Ensuite, nous parlerons des arbres de décision et découvrirons comment trouver de telles règles **automatiquement** uniquement sur la base des données d’entrée;
- Nous avons obtenu ces deux bases sans appliquer l’apprentissage automatique et elles serviront de point de départ pour nos modèles ultérieurs. S'il s'avère qu'avec un effort énorme, nous n'augmentons la précision que de 0,5%, alors nous avons peut-être commis une erreur, et il suffit de nous en tenir à un simple modèle "if-else" avec deux conditions;
- Avant de former des modèles complexes, il est recommandé de mélanger un peu les données, de tracer des graphiques et de vérifier des hypothèses simples. De plus, dans les applications métier de l'apprentissage automatique, on commence généralement par des solutions simples, pour ensuite expérimenter des solutions plus complexes.

## 3. Plus d'analyse visuelle

### Histogrammes et diagrammes de densité

La façon la plus simple d'examiner la distribution d'une variable numérique est de tracer son *histogramme* à l'aide de la méthode de `DataFrame` [`hist()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.hist.html).

In [ ]:
features = ['Total day minutes', 'Total intl calls']
df[features].hist(figsize=(10, 4));

Il existe également un autre moyen, souvent plus clair, de saisir la distribution: *diagrammes de densité* ou, plus officiellement, *diagrammes de densité de noyau*. Ils peuvent être considérés comme une version [lissée](https://en.wikipedia.org/wiki/Kernel_smoother) de l'histogramme. Leur principal avantage sur ces derniers est qu'ils ne dépendent pas de la taille des "_bins_". Créons des tracés de densité pour les deux mêmes variables:

In [ ]:
df[features].plot(kind='density', subplots=True, layout=(1, 2), 
                  sharex=False, figsize=(10, 4));

Il est également possible de tracer une distribution des observations avec la méthode [`distplot()`](https://seaborn.pydata.org/generated/seaborn.distplot.html) de de `seaborn`. Par exemple, regardons la distribution de `Total day minutes`. Par défaut, le tracé affiche à la fois l'histogramme avec [l'estimation de la densité du noyau](https://en.wikipedia.org/wiki/Kernel_estimation_de_la_densité_) (KDE) en haut.

In [ ]:
sns.distplot(df['Total day minutes']);

### Box plot (Boîte à moustaches)

Un autre type de visualisation utile est un *box plot*. `seaborn` fait un excellent travail ici:

In [ ]:
sns.boxplot(x='Total intl calls', data=df);

### Graphique en barres (Bar plot)

Le diagramme à barres est une représentation graphique de la table des fréquences. La façon la plus simple de le créer est d'utiliser la fonction de `seaborn`[`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html). Il existe une autre fonction dans «seaborn» qui est quelque peu confondue [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) et est principalement utilisée pour la représentation de certaines statistiques de base d'une variable numérique groupée par une caractéristique catégorielle.

Tracons les distributions de deux variables catégorielles:

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

sns.countplot(x='Churn', data=df, ax=axes[0]);
sns.countplot(x='Customer service calls', data=df, ax=axes[1]);

### Matrice de corrélation

Examinons les corrélations entre les variables numériques de notre ensemble de données. Ces informations sont importantes à connaître car il existe des algorithmes d'apprentissage automatique (par exemple, régression linéaire et logistique) qui ne gèrent pas bien les variables d'entrée hautement corrélées.

Tout d'abord, nous utiliserons la méthode [`corr()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) sur un `DataFrame` qui calcule la corrélation entre chaque paire de caratéristiques. Ensuite, nous passons la *matrice de corrélation* résultante à [`heatmap()`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) de `seaborn`, qui rend une une matrice à code couleur pour les valeurs fournies:

In [ ]:
## Correletion map avancé
def plot_correlation_map(df):
    corr = df.corr()
    _ , ax = plt.subplots(figsize =(24,20))
    cmap = sns.diverging_palette(220,10, as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={'shrink' : .9}, 
        ax=ax, 
        annot = True, 
        annot_kws = {'fontsize' : 12}
    )

In [ ]:
corr_matrix = df.drop(['State', 'International plan', 'Voice mail plan',
                      'Area code'], axis=1).corr()

In [ ]:
plot_correlation_map(corr_matrix)

À partir de la matrice de corrélation colorée générée ci-dessus, nous pouvons voir qu'il y a 4 variables telles que *Total day charge* qui ont été calculées directement à partir du nombre de minutes passées sur les appels téléphoniques (*Total day minutes*). Celles-ci sont appelées variables *dépendantes* et peuvent donc être omises car elles ne fournissent aucune information supplémentaire. 

### Nuage de points / Diagramme de dispersion (Scatter plot)

Le *nuage de points* affiche les valeurs de deux variables numériques en tant que *coordonnées cartésiennes* dans l'espace 2D. Des diagrammes de dispersion en 3D sont également possibles.

Essayons la fonction [`scatter()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.scatter.html) de la bibliothèque `matplotlib`:

In [ ]:
plt.scatter(df['Total day minutes'], df['Total night minutes']);

Nous obtenons une image sans intérêt de deux variables normalement distribuées. De plus, il semble que ces caractéristiques ne soient pas corrélées car la forme semblable à une ellipse est alignée avec les axes.

Il existe une option légèrement plus sophistiquée pour créer un nuage de points avec la bibliothèque `seaborn`:

In [ ]:
sns.jointplot(x='Total day minutes', y='Total night minutes', 
              data=df, kind='scatter');

#### Matrice de nuage de points (Scatterplot matrix)

Dans certains cas, nous pouvons vouloir tracer une *matrice de nuage de points* telle que celle illustrée ci-dessous. Sa diagonale contient les distributions des variables correspondantes et les diagrammes de dispersion pour chaque paire de variables remplissent le reste de la matrice.

In [ ]:
features = list(set(df.columns) - set(['State', 'International plan', 'Voice mail plan',  'Area code',
                                      'Total day charge',   'Total eve charge',   'Total night charge',
                                        'Total intl charge', 'Churn']))

In [ ]:
# `pairplot()` may become very slow with the SVG format
%config InlineBackend.figure_format = 'png'
sns.pairplot(df[features]);

De plus, les points peuvent être codés par couleur ou par taille afin que les valeurs d'une troisième variable catégorielle soient également présentées dans la même figure. On utilise le paramètre «hue» pour indiquer notre caractéristique d'intérêt:

In [ ]:
sns.pairplot(df[features + ['Churn']], hue='Churn')